Question : 

In [59]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-community langchain-openai tavily-python
%pip install langgraph langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core langchain-openai langchain-google-vertexai langchain-groq langchain-google-genai notebook tavily-python wikipedia pypdf chromadb


In [60]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="question3"

In [61]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

In [62]:


from typing import Annotated, Sequence
from typing_extensions import TypedDict
from typing import TypedDict, List, Optional


from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages

In [63]:
with open('question.txt', 'r') as file:
    question = file.read()
with open('model_solution.txt','r') as file:
    model_solution = file.read()
with open('rubric.txt','r') as file:
    rubric = file.read()
with open('student_solution.txt','r') as file:
    student_solution = file.read()

In [64]:
class ClassDetails(TypedDict):
    model_code: str
    student_code: str
    rubric: str
    initial_evaluation: str
    final_evaluation:  str 
    extracted_marks: str   

class AgentState(TypedDict):
    all_class_details: List[ClassDetails]


In [65]:
from pydantic import BaseModel, Field

In [66]:
from typing import List, Tuple

In [67]:

from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field


from langgraph.prebuilt import tools_condition


In [68]:
def class_extractor(state : AgentState):
	prompt = '''You are a class extractor. Given a student's code and a model solution, 
              your task is to identify the classes that are present in both the student's code and the model solution. 
              You should return a list of tuples where the first element is the class from the student's code, 
              and the second element is the corresponding class from the model solution.
			  model solution = {model_solution}
			  student solution = {student_solution}
             '''
	
	class extract(BaseModel):
		classes_code: List[Tuple[str, str]] = Field(
            description="First element of tuple is the class from student code and second element is the class from the model solution"
        )

	model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
	llm_with_tool = model.with_structured_output(extract)
	chain = prompt | llm_with_tool
	response = chain.invoke({"model_solution" : model_solution , "student_solution": student_solution})
	extracted_classes = response.extracted_classes  # This is a list of tuples (student_class, model_class)
	all_class_details_new = []
	for student_class, model_class in extracted_classes:
		class_detail = ClassDetails(
            model_code=model_class,
            student_code=student_class,
            rubric="",  # Populate as needed
            initial_evaluation="",  # Populate as needed
            final_evaluation="",  # Populate as needed
            extracted_marks=""  # Populate as needed
        )
		all_class_details_new.append(class_detail)
	
	return {'all_class_details': all_class_details_new} 


In [69]:
def rubric_extraction_module(state : AgentState): 
	all_class_details = state['all_class_details']
	prompt = '''
	Given the question {question} , rubric {rubric} your task is to generate the rubric for only a particular class of the code
	the model solution for the class will be provided {model_class}
	'''
	all_class_details_new = []
	for cls in all_class_details:
		model_code = cls['model_code']
		model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
		prompt_final = prompt.format(question = question,rubric=rubric, model_class  = model_code)
		response = model.invoke(prompt_final)
		class_detail = ClassDetails(
			model_code=model_code,
            student_code= cls['student_code'],
            rubric= response,  # Populate as needed
            initial_evaluation="",  # Populate as needed
            final_evaluation="",  # Populate as needed
            extracted_marks=""  # Populate as needed

		)
		all_class_details_new.append(class_detail)

	return {'all_class_details': all_class_details_new}

		
	

In [70]:
def initial_evaluation_module(state : AgentState):
	all_class_details = state['all_class_details']
	prompt = '''
	Given the rubric {rubric} and model solution {model_code} of a class you have to grade the student solution
	{student_code} for the class, I want you to give numerical score as well as comments on correctness, errors and suggestions 
	and improvement in student code
	'''
	all_class_details_new = []
	for cls in all_class_details:
		model_code = cls['model_code']
		student_code = cls['student_code']
		rubric = cls['rubric']
		final_prompt = prompt.format(rubric = rubric , model_code = model_code , student_code = student_code)
		model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
		response = model.invoke(prompt)
		class_detail = ClassDetails(
			model_code=model_code,
            student_code= student_code,
            rubric= rubric,  
            initial_evaluation= response, 
            final_evaluation="",
            extracted_marks="" 

		)
		all_class_details_new.append(class_detail)


In [71]:
def review_evaluation_module(state : AgentState):
	all_class_details = state['all_class_details']
	prompt = '''
	Given the rubric {rubric} and model solution {model_code} of a class you have to grade the student solution
	{student_code} for the class, as well as a first draft of the evaluation {initial_evaluation} , I want you to 
	make the evaluation better make sure it is in line with the rubric
	'''
	all_class_details_new = []
	for cls in all_class_details:
		model_code = cls['model_code']
		student_code = cls['student_code']
		rubric = cls['rubric']
		initial_evaluation = cls['initial_evaluation']
		final_prompt = prompt.format(rubric = rubric , model_code = model_code , student_code = student_code, initial_evaluation = initial_evaluation)
		model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
		response = model.invoke(prompt)
		class_detail = ClassDetails(
			model_code=model_code,
            student_code= student_code,
            rubric= rubric,  
            initial_evaluation= initial_evaluation, 
            final_evaluation= response,
            extracted_marks="" 

		)
		all_class_details_new.append(class_detail)

In [72]:
def marks_extraction_module(state : AgentState):
	all_class_details = state['all_class_details']
	prompt = '''
	Given a evaluation, I want you to extract all the marks in the evaluation {final_evaluation}, give the marks in a comma separated 
	string.
	'''
	all_class_details_new = []
	for cls in all_class_details:
		model_code = cls['model_code']
		student_code = cls['student_code']
		rubric = cls['rubric']
		initial_evaluation = cls['initial_evaluation']
		final_evaluation = cls['final_evaluation']
		final_prompt = prompt.format(final_evaluation = final_evaluation)
		model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
		response = model.invoke(prompt)
		class_detail = ClassDetails(
			model_code=model_code,
            student_code= student_code,
            rubric= rubric,  
            initial_evaluation= initial_evaluation, 
            final_evaluation= final_evaluation,
            extracted_marks= response

		)
		all_class_details_new.append(class_detail)

In [73]:
def sum_marks(comma_separated_integers):
    # Split the string by commas and convert each element to an integer
    integers_list = [int(x) for x in comma_separated_integers.split(',')]
    # Return the sum of the integers in the list
    return sum(integers_list)


In [74]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

In [89]:
workflow = StateGraph(AgentState)
workflow.add_node("class_extractor", class_extractor)
workflow.add_node("rubric_extraction_module", rubric_extraction_module)
workflow.add_node("initial_evaluation_module", initial_evaluation_module)
workflow.add_node("review_evaluation_module", review_evaluation_module)
workflow.add_node("marks_extraction_module", marks_extraction_module)

workflow.add_edge(START, "class_extractor")
workflow.add_edge("class_extractor", "rubric_extraction_module")
workflow.add_edge("rubric_extraction_module", "initial_evaluation_module")
workflow.add_edge("initial_evaluation_module", "review_evaluation_module")
workflow.add_edge("initial_evaluation_module", "marks_extraction_module")
workflow.add_edge("marks_extraction_module", END)


graph = workflow.compile()

In [88]:

from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [86]:
# prompt: give code to call the agent
import pprint

initial_state = {
    'all_class_details' : []
}
final_output = None
for output in graph.stream(initial_state):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=30, depth=None)
    pprint.pprint("\n---\n")

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'all_class_details' : []
}, thread):
    print(s)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>